# Nutritional Inequality and Food Prices  

## Introduction  

Nutritional inequality refers to disparities in diet quality and access to healthy food across different socioeconomic groups. Rising food prices, food deserts, and lack of nutrition education contribute to this issue, affecting millions of Americans. As of 2025, **around 40 million Americans** experience food insecurity, with many turning to cheaper, less nutritious options due to affordability and accessibility challenges.  

In this project, we explore **how food prices impact dietary choices and accessibility across different regions and income levels**. Our research focuses on:  

- **Comparing** the cost of nutrient-dense foods (fruits, vegetables) versus processed foods  
- **Analyzing** geographical disparities in food availability (food deserts)  
- **Investigating** the role of socioeconomic factors in nutritional choices 

To address these questions, we leverage publicly available datasets from **USDA, BLS, FAO, and Kaggle**, integrating statistical analysis and visualization techniques. Our goal is to uncover meaningful insights into the systemic causes of nutritional inequality and potential policy interventions.  

---  
### Notebook Workflow  
🔹 **Data Wrangling:** Cleaning and preprocessing datasets  
🔹 **Combining Data:** Merging multiple sources for comprehensive analysis  
🔹 **Statistical Analysis:** Identifying correlations between food prices, income, and access  
🔹 **Visualization:** Presenting trends and disparities through charts and maps  

This notebook serves as a prototype for our analysis, demonstrating key data transformations, exploratory statistics, and preliminary findings. 🚀  

In [2]:
# Load in packages
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import geopandas as gpd

# others

ModuleNotFoundError: No module named 'geopandas'

In [3]:
# Load in datasets
folder_path = 'project data/'

historical_changes_consumer_price = pd.read_csv(folder_path + '(historical) changes_consumer_price.csv')
historical_producer_price_changes = pd.read_csv(folder_path + '(historical) producer_price_changes.csv')
state_and_county = pd.read_csv(folder_path + 'StateAndCountyData.csv')
variable_list = pd.read_csv(folder_path + 'VariableList.csv')

## Data Wrangling 

### For CPI + PPI 

In [5]:
historical_producer_price_changes.head()

,Producer Price Index item,Year,Percent change
0,Unprocessed foodstuffs and feedstuffs,1974,5.2
1,Unprocessed foodstuffs and feedstuffs,1975,1.3
2,Unprocessed foodstuffs and feedstuffs,1976,-0.8
3,Unprocessed foodstuffs and feedstuffs,1977,0.9
4,Unprocessed foodstuffs and feedstuffs,1978,12.6


In [6]:
## Showing data
historical_changes_consumer_price.head()

,Consumer Price Index item,Year,Percent change
0,All food,1974,14.3
1,All food,1975,8.5
2,All food,1976,3.0
3,All food,1977,6.3
4,All food,1978,9.9


In [4]:
# cleaning historical producer price changes + consumer price 

historical_producer_price_changes['Year'] = pd.to_numeric(historical_producer_price_changes['Year'], errors='coerce')
historical_producer_price_changes['Percent change'] = pd.to_numeric(historical_producer_price_changes['Percent change'], errors='coerce')
historical_changes_consumer_price['Year'] = pd.to_numeric(historical_changes_consumer_price['Year'], errors='coerce')
historical_changes_consumer_price['Percent change'] = pd.to_numeric(historical_changes_consumer_price['Percent change'], errors='coerce')

historical_producer_price_changes = historical_producer_price_changes.dropna()
historical_changes_consumer_price = historical_changes_consumer_price.dropna()

def categorize_food(item):
    item = item.lower()
    if "farm-level" in item:
        return "Farm Level"
    elif "wholesale" in item:
        return "Wholesale"
    elif "unprocessed" in item:
        return "Unprocessed"
    elif "processed" in item:
        return "Processed"
    elif "finished" in item:
        return "Finished"
    else:
        return "Other"

# create new category 
historical_producer_price_changes['Category'] = historical_producer_price_changes['Producer Price Index item'].apply(categorize_food)

print(historical_changes_consumer_price['Consumer Price Index item'].unique())

print(historical_producer_price_changes['Producer Price Index item'].unique())

# creating map as categories differ 
mapping = {
    "Wholesale beef": "Beef and veal",
    "Farm-level cattle": "Beef and veal",
    "Wholesale pork": "Pork",
    "Wholesale poultry": "Poultry",
    "Farm-level eggs": "Eggs",
    "Wholesale dairy": "Dairy products",
    "Farm-level milk": "Dairy products",
    "Farm-level soybeans": "Fats and oils",
    "Wholesale fats and oils": "Fats and oils",
    "Farm-level fruit": "Fresh fruits",
    "Farm-level vegetables": "Fresh vegetables",
    "Farm-level wheat": "Cereals and bakery products",
    "Wholesale wheat flour": "Cereals and bakery products",
    "Unprocessed foodstuffs and feedstuffs": "All food",
    "Processed foods and feeds": "All food",
    "Finished consumer foods": "All food",
}

historical_producer_price_changes['Mapped Category'] = historical_producer_price_changes['Producer Price Index item'].map(mapping)

# merge data set 

cpi_ppi = pd.merge(
    historical_changes_consumer_price,
    historical_producer_price_changes,
    left_on=['Year', 'Consumer Price Index item'],
    right_on=['Year', 'Mapped Category'],
    suffixes=('_consumer', '_producer')
)

# remove columns 

cpi_ppi = cpi_ppi.drop(columns=['Producer Price Index item', 'Mapped Category'])
cpi_ppi = cpi_ppi.dropna()

# display 

print(cpi_ppi.head())

['All food' 'Food away from home' 'Food at home'
 'Meats, poultry, and fish' 'Meats' 'Beef and veal' 'Pork' 'Other meats'
 'Poultry' 'Fish and seafood' 'Eggs' 'Dairy products' 'Fats and oils'
 'Fruits and vegetables' 'Fresh fruits and vegetables' 'Fresh fruits'
 'Fresh vegetables' 'Processed fruits and vegetables' 'Sugar and sweets'
 'Cereals and bakery products' 'Nonalcoholic beverages' 'Other foods']
['Unprocessed foodstuffs and feedstuffs' 'Processed foods and feeds'
 'Finished consumer foods' 'Farm-level cattle' 'Wholesale beef'
 'Wholesale pork' 'Wholesale poultry' 'Farm-level eggs' 'Farm-level milk'
 'Wholesale dairy' 'Farm-level soybeans' 'Wholesale fats and oils'
 'Farm-level fruit' 'Farm-level vegetables' 'Farm-level wheat'
 'Wholesale wheat flour']
  Consumer Price Index item  Year  Percent change_consumer  \
0                  All food  1974                     14.3   
1                  All food  1974                     14.3   
2                  All food  1974            

### Stat + County Data Set

In [9]:
## Variable list
variable_list.head(10)

,Variable_Name,Category_Name,Category_Code,Subcategory_Name,Variable_Code,Geography,Units
0,"Population, low access to store, 2010",Access and Proximity to Grocery Store,ACCESS,Overall,LACCESS_POP10,CNTY10,Count
1,"Population, low access to store, 2015",Access and Proximity to Grocery Store,ACCESS,Overall,LACCESS_POP15,CNTY10,Count
2,"Population, low access to store (% change), 20...",Access and Proximity to Grocery Store,ACCESS,Overall,PCH_LACCESS_POP_10_15,CNTY10,% change
3,"Population, low access to store (%), 2010",Access and Proximity to Grocery Store,ACCESS,Overall,PCT_LACCESS_POP10,CNTY10,Percent
4,"Population, low access to store (%), 2015",Access and Proximity to Grocery Store,ACCESS,Overall,PCT_LACCESS_POP15,CNTY10,Percent
5,"Low income & low access to store, 2010",Access and Proximity to Grocery Store,ACCESS,Household Resources,LACCESS_LOWI10,CNTY10,Count
6,"Low income & low access to store, 2015",Access and Proximity to Grocery Store,ACCESS,Household Resources,LACCESS_LOWI15,CNTY10,Count
7,"Low income & low access to store (% change), 2...",Access and Proximity to Grocery Store,ACCESS,Household Resources,PCH_LACCESS_LOWI_10_15,CNTY10,% change
8,"Low income & low access to store (%), 2010",Access and Proximity to Grocery Store,ACCESS,Household Resources,PCT_LACCESS_LOWI10,CNTY10,Percent
9,"Low income & low access to store (%), 2015",Access and Proximity to Grocery Store,ACCESS,Household Resources,PCT_LACCESS_LOWI15,CNTY10,Percent


In [10]:
## State + County Data set 
state_and_county.head()

,FIPS,State,County,Variable_Code,Value
0,1001,AL,Autauga,LACCESS_POP10,18428.439690
1,1001,AL,Autauga,LACCESS_POP15,17496.693040
2,1001,AL,Autauga,PCH_LACCESS_POP_10_15,-5.056026
3,1001,AL,Autauga,PCT_LACCESS_POP10,33.769657
4,1001,AL,Autauga,PCT_LACCESS_POP15,32.062255


In [11]:
#merging with variable code for ease of understanding
geog_food = state_and_county.merge(variable_list, on="Variable_Code", how="left")

## Statistical Analysis

In [13]:
## hypothesis test

## Visualization

In [5]:
#chloropeth maps!!

shapefile = gpd.read_file("cb_2018_us_state_20m/cb_2018_us_state_20m.shp")

state_geog_food = shapefile.merge(geog_food, left_on="STUSPS", right_on="State")

state_subset_2015 = state_geog_food[(state_geog_food["Variable_Code"] == "PCT_LACCESS_POP15") &
                                (~state_geog_food["STUSPS"].isin(["AK", "HI", "PR"]))]

#2015 LACCESS percent
fig, ax = plt.subplots(1, figsize=(12, 12))
ax.axis('off')
ax.set_title('Percent Population with Limited Access to Grocery Stores (2015)',
             fontdict={'fontsize': '15', 'fontweight' : '3'})
fig = state_subset_2015.plot(column='Value', cmap='YlOrRd', linewidth=0.5, ax=ax, edgecolor='0.2',legend=True)

#2010 LACCESS percent
state_subset_2010 = state_geog_food[(state_geog_food["Variable_Code"] == "PCT_LACCESS_POP10") &
                                (~state_geog_food["STUSPS"].isin(["AK", "HI", "PR"]))]

fig, ax = plt.subplots(1, figsize=(12, 12))
ax.axis('off')
ax.set_title('Percent Population with Limited Access to Grocery Stores (2010)',
             fontdict={'fontsize': '15', 'fontweight' : '3'})
fig = state_subset_2010.plot(column='Value', cmap='YlOrRd', linewidth=0.5, ax=ax, edgecolor='0.2',legend=True)

NameError: name 'gpd' is not defined

Using the GeoPandas library, we created a choropleth map of the United States to visualize food insecurity across different states, tracking geographical disparities in access to grocery stores in 2015 and 2010.

food desert:
PCT_LACCESS_POP15 > 30% → High % of people with limited access
PCH_LACCESS_POP_10_15 >= 0 → No improvement or worsening from 2010 to 2015

- Add another column based on clustering counties together? politics, coast, metropolitian etc
- `is_food_desert` : PCT_LACCESS_POP15 > 30
- `improved` : PCH_LACCESS_POP_10_15 >= 0
- `needs_help` : `is_food_desert` == 1 & `improved` == 0

Logistic regression model to predict whether a county needs help or not

- response var: `needs_help`
- predictors: 

goal: A shortlist of counties (FIPS, name, score) most in need

In [8]:
state_and_county_mod = state_and_county[state_and_county['County'] != 'Total']

state_and_county_cleaned = state_and_county_mod.pivot_table(
    index=['FIPS', 'State', 'County'],
    columns="Variable_Code",
    values="Value"
)

state_and_county["Variable_Code"].unique()

#percent low access, percent low access & low income, percent
geog_vars = ['PCH_LACCESS_POP_10_15','PCT_LACCESS_POP10', 'PCT_LACCESS_POP15',
             'PCH_LACCESS_LOWI_10_15', 'PCT_LACCESS_LOWI10','PCT_LACCESS_LOWI15',
            'METRO13', 'POVRATE15']

state_county_filtered = state_and_county_cleaned[geog_vars].dropna()

state_county_filtered.head(10)

#drop 2010 census pop col
#remove county : '. County'
#predict based on regions/ counties?

,,Variable_Code,PCH_LACCESS_POP_10_15,PCT_LACCESS_POP10,PCT_LACCESS_POP15,PCH_LACCESS_LOWI_10_15,PCT_LACCESS_LOWI10,PCT_LACCESS_LOWI15,METRO13,POVRATE15
FIPS,State,County,,,,,,,,
1001,AL,Autauga,-5.056026,33.769657,32.062255,22.439248,9.793530,11.991125,1.0,12.7
1003,AL,Baldwin,-13.204891,19.318473,16.767489,-0.656270,5.460261,5.424427,1.0,12.9
1005,AL,Barbour,6.067799,20.840972,22.105560,-5.959985,11.420316,10.739667,0.0,32.0
1007,AL,Bibb,-7.224696,4.559753,4.230324,21.307144,2.144661,2.601627,1.0,22.2
1009,AL,Blount,140.568857,2.700840,6.497380,171.081177,1.062468,2.880150,1.0,14.7
1011,AL,Bullock,1.269365,37.474652,37.950342,-2.026523,20.151730,19.743351,0.0,39.6
1013,AL,Butler,-0.266929,6.243470,6.226805,3.322452,2.832029,2.926122,0.0,25.8
1015,AL,Calhoun,-10.081061,26.061086,23.433852,-8.614820,10.043824,9.178567,1.0,20.0
1017,AL,Chambers,5.679911,19.722967,20.843214,17.083593,8.821525,10.328558,0.0,22.4


In [ ]:
# def food_desert_val(row):
#     if row["PCT_LACCESS_POP15"]["Value"] > 30 | row["PCT_LACCESS_POP10"] > 30:
#         val = 1
#     else:
#         val = 0
#     return val

# state_and_county_cleaned["is_food_desert"] = state_and_county_cleaned.apply(food_desert_val, axis = 1)

# state_and_county_cleaned.head()

In [ ]:
# TRIED a smaller version
fig, axes = plt.subplots(1, 2, figsize=(20, 10))
for ax, year, title in zip(axes, 
                           ['PCT_LACCESS_POP10', 'PCT_LACCESS_POP15'], 
                           ['2010', '2015']):
    subset = state_geog_food[(state_geog_food["Variable_Code"] == year) & 
                             (~state_geog_food["STUSPS"].isin(["AK", "HI", "PR"]))]
    ax.axis('off')
    ax.set_title(f'% Limited Grocery Access ({title})', fontsize=15)
    subset.plot(column='Value', cmap='YlOrRd', linewidth=0.5, ax=ax, edgecolor='0.2', legend=True)
plt.tight_layout()
plt.show()

In [ ]:
# Aggregate to state level
top_states = state_and_county[state_and_county['Variable_Code'] == 'PCT_LACCESS_POP15'] \
                .groupby('State')['Value'].mean().sort_values(ascending=False).head(10)

# Plot
plt.figure(figsize=(10, 6))
sns.barplot(x=top_states.values, y=top_states.index, hue=top_states.index, palette='Reds_r')
plt.title('Top 10 States with Highest % Population with Low Grocery Access (2015)')
plt.xlabel('Average % Low Access')
plt.ylabel('State')
plt.tight_layout()
plt.show()

In [ ]:
region_map = {
    # Northeast
    'CT': 'Northeast',
    'ME': 'Northeast',
    'MA': 'Northeast',
    'NH': 'Northeast',
    'RI': 'Northeast',
    'VT': 'Northeast',
    'NJ': 'Northeast',
    'NY': 'Northeast',
    'PA': 'Northeast',

    # Midwest
    'IL': 'Midwest',
    'IN': 'Midwest',
    'MI': 'Midwest',
    'OH': 'Midwest',
    'WI': 'Midwest',
    'IA': 'Midwest',
    'KS': 'Midwest',
    'MN': 'Midwest',
    'MO': 'Midwest',
    'NE': 'Midwest',
    'ND': 'Midwest',
    'SD': 'Midwest',

    # South
    'DE': 'South',
    'DC': 'South',
    'FL': 'South',
    'GA': 'South',
    'MD': 'South',
    'NC': 'South',
    'SC': 'South',
    'VA': 'South',
    'WV': 'South',
    'AL': 'South',
    'KY': 'South',
    'MS': 'South',
    'TN': 'South',
    'AR': 'South',
    'LA': 'South',
    'OK': 'South',
    'TX': 'South',

    # West
    'AZ': 'West',
    'CO': 'West',
    'ID': 'West',
    'MT': 'West',
    'NV': 'West',
    'NM': 'West',
    'UT': 'West',
    'WY': 'West',
    'AK': 'West',
    'CA': 'West',
    'HI': 'West',
    'OR': 'West',
    'WA': 'West',
}

state_and_county['Region'] = state_and_county['State'].map(region_map)

region_data = state_and_county[state_and_county['Variable_Code'] == 'PCT_LACCESS_POP10'].dropna(subset=['Region'])

plt.figure(figsize=(8, 6))
sns.boxplot(data=region_data, x='Region', y='Value', hue='Region', palette='Set2')
plt.title('Distribution of Low Grocery Access by Region (2010)')
plt.ylabel('% Low Access')
plt.xlabel('Region')
plt.tight_layout()
plt.show()

region_data = state_and_county[state_and_county['Variable_Code'] == 'PCT_LACCESS_POP15'].dropna(subset=['Region'])

plt.figure(figsize=(8, 6))
sns.boxplot(data=region_data, x='Region', y='Value', hue='Region', palette='Set2')
plt.title('Distribution of Low Grocery Access by Region (2015)')
plt.ylabel('% Low Access')
plt.xlabel('Region')
plt.tight_layout()
plt.show()

In [ ]:
# for use in analysis

party_map = {
    "AL": "Red",
    "AR": "Red",
    "FL": "Red",
    "GA": "Red",
    "ID": "Red",
    "IN": "Red",
    "IA": "Red",
    "KS": "Red",
    "KY": "Red",
    "LA": "Red",
    "MS": "Red",
    "MO": "Red",
    "MT": "Red",
    "NE": "Red",
    "NC": "Red",
    "ND": "Red",
    "OH": "Red",
    "OK": "Red",
    "SC": "Red",
    "SD": "Red",
    "TN": "Red",
    "TX": "Red",
    "UT": "Red",
    "WV": "Red",
    "WI": "Red",
    "WY": "Red",
    "AZ": "Red",
    "MI": "Red",
    "PA": "Red",

    # Blue states
    "CA": "Blue",
    "CO": "Blue",
    "IL": "Blue",
    "MD": "Blue",
    "MA": "Blue",
    "NJ": "Blue",
    "NY": "Blue",
    "OR": "Blue",
    "RI": "Blue",
    "VT": "Blue",
    "VA": "Blue",
    "WA": "Blue",
    "NM": "Blue",
    "CT": "Blue",
    "DE": "Blue",
    "DC": "Blue",
    "MN": "Blue",
    "ME": "Blue",
    "NH": "Blue"
}

In [ ]:
# Calculate change from 2010 to 2015
slope = state_and_county[state_and_county['Variable_Code'].isin(['PCT_LACCESS_POP10', 'PCT_LACCESS_POP15'])]
slope = slope.pivot_table(index='State', columns='Variable_Code', values='Value').dropna()
slope = slope.sort_values('PCT_LACCESS_POP15', ascending=False)

# Normalize vertical space
slope = slope.reset_index()

slope['Change'] = slope['PCT_LACCESS_POP15'] - slope['PCT_LACCESS_POP10']

# Top 10 improvement (negative change = better access)
top_improve = slope.sort_values('Change').head(10)

# Top 10 worsening (positive change = worse access)
top_worsen = slope.sort_values('Change', ascending=False).head(10)

# Plot side-by-side
fig, axes = plt.subplots(1, 2, figsize=(16, 6), sharex=True)

# Improvements
sns.barplot(x='Change', y='State', hue='State', data=top_improve, ax=axes[0], palette='Greens')
axes[0].set_title('Top 10 States - Improved Grocery Access (2010–2015)')
axes[0].set_xlabel('Change in % Low Access')
axes[0].set_ylabel('')

# Worsening
sns.barplot(x='Change', y='State', hue='State', data=top_worsen, ax=axes[1], palette='Reds')
axes[1].set_title('Top 10 States - Worsened Grocery Access (2010–2015)')
axes[1].set_xlabel('Change in % Low Access')
axes[1].set_ylabel('')

plt.tight_layout()
plt.show()

In [9]:
# Add region info
bars_2015 = state_and_county[state_and_county['Variable_Code'].isin(['PCT_LACCESS_POP15', 'PCT_LACCESS_LOWI15'])]

bars_2015_pivot = bars_2015.pivot(index=['FIPS', 'State', 'County'], 
                                  columns='Variable_Code', 
                                  values='Value').reset_index()

bars_2015_pivot['Region'] = bars_2015_pivot['State'].map(region_map)

# Filter out any rows with missing regions
bars_2015_pivot = bars_2015_pivot.dropna(subset=['Region'])

# Group by region, calculating average % for both categories
region_avg = bars_2015_pivot.groupby('Region')[['PCT_LACCESS_POP15', 'PCT_LACCESS_LOWI15']].mean()

# Plot
region_avg.plot(kind='bar', figsize=(12, 7), color=['#1f77b4', '#ff7f0e'])
plt.title('Average Percent with Limited Access by Region (2015)', fontsize=14)
plt.ylabel('Percent of Population')
plt.xlabel('Region')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

NameError: name 'region_map' is not defined

In [ ]:
# Add region info to pivot data
bars_2015_pivot['Region'] = bars_2015_pivot['State'].map(region_map)

# Calculate average access for each region
region_avg = bars_2015_pivot.groupby('Region')[['PCT_LACCESS_POP15', 'PCT_LACCESS_LOWI15']].mean().reset_index()

# Scatter plot by region (color by region)
plt.figure(figsize=(10, 7))
sns.scatterplot(data=region_avg, 
                x='PCT_LACCESS_POP15', 
                y='PCT_LACCESS_LOWI15', 
                hue='Region', 
                palette='Set2', 
                s=100,  # Size of the points
                alpha=0.8)

# Title and labels
plt.title('Low-Income Access vs Total Population Access by Region (2015)', fontsize=14)
plt.xlabel('% Total Population with Low Access', fontsize=12)
plt.ylabel('% Low-Income Population with Low Access', fontsize=12)

# Add 1:1 line (perfect correlation)
plt.axline((0, 0), slope=1, color='gray', linestyle='--', linewidth=1)

# Clean up legend
plt.legend(title='Region', bbox_to_anchor=(1.05, 1), loc='upper left')

# Display grid
plt.grid(True)

# Layout adjustments
plt.tight_layout()
plt.show()

In [ ]:
# COULD do the same with 2010

plt.figure(figsize=(10, 6))
sns.histplot(data=bars_2015_pivot, x='PCT_LACCESS_POP15', bins=30, kde=True)
plt.title('Distribution of % Population with Limited Access to Grocery Stores (2015)')
plt.xlabel('% Population with Low Access')
plt.ylabel('Number of Counties')
plt.tight_layout()
plt.show()

In [ ]:
state_subset_change = state_geog_food[(state_geog_food["Variable_Code"] == "PCH_LACCESS_POP_10_15") &
                                      (~state_geog_food["STUSPS"].isin(["AK", "HI", "PR"]))]

state_subset_2015 = state_geog_food[(state_geog_food["Variable_Code"] == "PCT_LACCESS_POP15") &
                                (~state_geog_food["STUSPS"].isin(["AK", "HI", "PR"]))]

# Calculate IQR (Interquartile Range) and define vmin/vmax
q25, q75 = np.percentile(state_subset_change['Value'], 25), np.percentile(state_subset_change['Value'], 75)
iqr = q75 - q25
vmin = q25 - 1.5 * iqr
vmax = q75 + 1.5 * iqr

# Plot with adjusted color scale 
fig, ax = plt.subplots(1, figsize=(12, 12))
ax.axis('off')
ax.set_title('% Change in Population with Limited Access to Grocery Stores (2010–2015)',
             fontdict={'fontsize': '15', 'fontweight': '3'})

fig = state_subset_change.plot(column='Value', cmap='coolwarm', linewidth=0.5, ax=ax, edgecolor='0.2', legend=True,
                               vmin=vmin, vmax=vmax)

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
## Producer Data Visualization 

print("Producer Data Summary:")
print(historical_producer_price_changes.head())
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as mpatches

# Make sure your data is loaded into this DataFrame
# historical_producer_price_changes = pd.read_csv(...) or however you're loading it

# Set style
sns.set(style="whitegrid")

# Sort food types to keep legend clean and consistent
unique_food_types = sorted(historical_producer_price_changes["Producer Price Index item"].unique())

# Assign colors using tab20 palette
palette = sns.color_palette("tab20", len(unique_food_types))
color_dict = dict(zip(unique_food_types, palette))

# Create the FacetGrid (no legend from here)
g = sns.FacetGrid(
    historical_producer_price_changes,
    col="Category",
    col_wrap=3,
    height=5,
    aspect=1.2,
    hue="Producer Price Index item",
    palette=color_dict,
    sharey=False,
    legend_out=True  # turn off auto-legend
)

# Scatter plots
g.map_dataframe(
    sns.scatterplot,
    x="Year",
    y="Percent change",
    alpha=0.7,
    s=60
)

# Add regression lines per subplot
for ax, category in zip(g.axes.flat, historical_producer_price_changes['Category'].unique()):
    data = historical_producer_price_changes[
        historical_producer_price_changes['Category'] == category
    ]
    sns.regplot(
        data=data,
        x="Year",
        y="Percent change",
        scatter=False,
        ax=ax,
        color='black',
        ci=95,
        line_kws={'linewidth': 2, 'linestyle': '--'}
    )
    ax.set_title(category, fontsize=14)
    sns.despine(ax=ax)

# Axis and layout adjustments
g.set_axis_labels("Year", "Percent Change (%)")
g.set_titles("{col_name}")
g.fig.subplots_adjust(top=0.92, bottom=0.1, left=0.05, right=0.8, hspace=0.4, wspace=0.3)
g.fig.suptitle("Producer Price Changes Over Time by Category and Food Type", fontsize=20)

# Manually build legend patches
handles = [
    mpatches.Patch(color=color_dict[ftype], label=ftype)
    for ftype in unique_food_types
]

# Add full manual legend
g.fig.legend(
    handles=handles,
    title="Food Type",
    loc='lower center',
    bbox_to_anchor=(0.5, -0.10),  # push this value down as much as you want
    frameon=True,
    fontsize=10,
    title_fontsize=12,
    ncol=4  # you can increase this if you want a tighter, wider legend
)


plt.show()


While this visualiation does not reveal a strong linear relationship between Producer Price Changes Over Time when faceted by Category and Food Type, it suggests that generally Producer Price Index percentage changes for Farm Level and Wholesale Products experience more drastic changes compared to Processed. This could suggest that perhaps Processed Foods experience less drastic changes in profits for producers, and thus could drive for preference for Processed Foods from a Producer's standpoint as the Producer Price Index appears more stable. 

In [ ]:
## PPI vs CPI 

g = sns.FacetGrid(cpi_ppi, col="Category", col_wrap=3, height=5, sharex=False, sharey=False)
g.map_dataframe(sns.scatterplot, x="Percent change_consumer", y="Percent change_producer")

# Adding titles and labels
g.set_axis_labels("Consumer Price Index (CPI) Percent Change", "Producer Price Index (PPI) Percent Change")
g.set_titles("{col_name}")

plt.subplots_adjust(top=0.9)
g.fig.suptitle("CPI vs PPI by Category")

for ax, category in zip(g.axes.flat, cpi_ppi["Category"].unique()):
    data = cpi_ppi[cpi_ppi["Category"] == category]
    sns.regplot(data=data, x="Percent change_consumer", y="Percent change_producer", ax=ax, 
                scatter=False, color='red', ci=None, line_kws={'linewidth': 2})

from scipy.stats import pearsonr

for ax, category in zip(g.axes.flat, cpi_ppi["Category"].unique()):
    data = cpi_ppi[cpi_ppi["Category"] == category]
    corr, _ = pearsonr(data["Percent change_consumer"], data["Percent change_producer"])
    ax.annotate(f"r = {corr:.2f}", xy=(0.05, 0.85), xycoords="axes fraction", fontsize=12, fontweight="bold", color="black")

g.map_dataframe(sns.scatterplot, x="Percent change_consumer", y="Percent change_producer", alpha=0.6, hue="Category", palette="Set2")

plt.show()


From this visualization, we can see that for Farm Level nd Finished Products, theres a clear positive moderately strong linear relationship between CPI Change and PPI Change, however, the slope is difficult to compare due to differing interval values. In this data set, "Finished" refers to packaged, ready to eat goods like cookies, snacks, etc. On the other hand, farm level is typical raw products like meats, produce, etc. sourced directly from farms. Thus, our next step would be to create a linear regression model to directly understand these slopes and perhaps group "Finished" and "Processed" products into an "Unhealthy" food category, and "Farm Level" and "Unprocessed" together into a Healthy category. 

In [ ]:
# from scipy.stats import pearsonr

# # Set style
# sns.set(style="whitegrid")

# # Define colors for health categories
# colors = {"Healthy": "green", "Unhealthy": "red", "Other": "gray"}

# # Create the facet grid
# g = sns.FacetGrid(cpi_ppi, col="Health Category", height=6, sharex=True, sharey=True, palette=colors)
# g.map_dataframe(sns.scatterplot, x="Percent change_consumer", y="Percent change_producer", alpha=0.6, edgecolor="black")

# # Add regression lines & correlation coefficients
# for ax, category in zip(g.axes.flat, cpi_ppi['Health Category'].unique()):
#     data = cpi_ppi[cpi_ppi['Health Category'] == category]
    
#     # Regression line with confidence interval
#     sns.regplot(data = data, x = "Percent change_consumer", y = "Percent change_producer", ax = ax, 
#                 scatter = False, color=colors[category], ci = 95, line_kws = {'linewidth': 2})
    
#     # calculate Pearson correlation
#     corr, _ = pearsonr(data['Percent change_consumer'], data['Percent change_producer'])
#     ax.text(0.05, 0.9, f"r = {corr:.2f}", transform=ax.transAxes, fontsize=12, fontweight="bold", color="black")

# # Adjust titles & labels
# g.set_axis_labels("CPI Percent Change", "PPI Percent Change")
# g.set_titles("{col_name}")

# # Improve title positioning
# plt.subplots_adjust(top=0.85)
# g.fig.suptitle("CPI vs PPI - Healthy vs Unhealthy Foods", fontsize=16, fontweight="bold")

# # Show the figure
# plt.show()

From here, we decided to look at the linear regression models directly. 

In [ ]:
## linear Regression Models 

# categories = cpi_ppi['Health Category'].unique()

# for category in categories:

#     data = cpi_ppi[cpi_ppi['Health Category'] == category]

#     slope, intercept, r_value, p_value, std_err = stats.linregress(
#         data['Percent change_consumer'],
#         data['Percent change_producer']
#     )

#     # Print results
#     print(f"Category: {category}")
#     print(f"  Slope: {slope:.4f}")
#     print(f"  Intercept: {intercept:.4f}")
#     print(f"  R-squared: {r_value**2:.4f}")
#     print(f"  P-value: {p_value:.4e}")
#     print(f"  Standard Error: {std_err:.4f}")
#     print("-" * 50)

Our linear regression models suggest that for Healthy foods, for every one percent increase in CPI, we would expect, on average, for the PPI to increase by around 1.53%, while for Unhealthy foods, for every one percent increase in CPI, we would expect, on average, for the PPI to increase by 1.13%. This could possibly suggest that producers make more profit when the price for consumers are increased for Healthy foods compared to Unhealthy foods, and thus producers may be incentivized to increase the price of Healthy foods over those of Unhealthy. 

In [ ]:
# sns.set(style="whitegrid")

# # calculate statistics 
# median_healthy = cpi_ppi[cpi_ppi['Health Category'] == 'Healthy']['Percent change_consumer'].median()
# median_unhealthy = cpi_ppi[cpi_ppi['Health Category'] == 'Unhealthy']['Percent change_consumer'].median()
# mean_healthy = cpi_ppi[cpi_ppi['Health Category'] == 'Healthy']['Percent change_consumer'].mean()
# mean_unhealthy = cpi_ppi[cpi_ppi['Health Category'] == 'Unhealthy']['Percent change_consumer'].mean()

# # figure size
# plt.figure(figsize=(12, 6))

# # healthy foods histogram
# plt.subplot(1, 2, 1)
# sns.histplot(data=cpi_ppi[cpi_ppi['Health Category'] == 'Healthy'], 
#              x='Percent change_consumer', 
#              kde=False, 
#              bins=20, 
#              color='green')

# # add median and mean indicators
# plt.axvline(median_healthy, color='black', linestyle='dashed', linewidth=1.5, label=f'Median: {median_healthy:.2f}')
# plt.axvline(mean_healthy, color='red', linestyle='dotted', linewidth=1.5, label=f'Mean: {mean_healthy:.2f}')
# plt.legend()

# plt.title('Consumer Price Index (CPI) - Healthy Foods')
# plt.xlabel('CPI Percent Change')
# plt.ylabel('Frequency')

# # unhealthy foods histogram
# plt.subplot(1, 2, 2)
# sns.histplot(data=cpi_ppi[cpi_ppi['Health Category'] == 'Unhealthy'], 
#              x='Percent change_consumer', 
#              kde=False, 
#              bins=20, 
#              color='blue')

# # Find global min and max for x-axis limits
# global_min = cpi_ppi['Percent change_consumer'].min()
# global_max = cpi_ppi['Percent change_consumer'].max()
# plt.xlim(global_min, global_max)

# plt.title('Consumer Price Index (CPI) - Unhealthy Foods')
# plt.xlabel('CPI Percent Change')
# plt.ylabel('Frequency')

# plt.tight_layout()
# plt.show()

In [ ]:
# Maybe disregard- unsure why household food insecurity is negative 

# Household food insecurity (%), three-year average, 2015-17 vs Population, low access to store, 2015

# Filter low access population data
filtered_data_low_access = state_and_county[state_and_county['Variable_Code'] == 'LACCESS_POP15'].copy()
filtered_data_low_access['Low Access Population'] = pd.to_numeric(filtered_data_low_access['Value'], errors='coerce')

state_avg_low_access = filtered_data_low_access.groupby('State')['Low Access Population'].mean().reset_index()

# Filter household insecurity data
filtered_data_insecurity = state_and_county[state_and_county['Variable_Code'] == 'FOODINSEC_15_17'][['County', 'State', 'Value']].copy()
filtered_data_insecurity.rename(columns={'Value': 'Household Insecurity'}, inplace=True)
filtered_data_insecurity['Household Insecurity'] = pd.to_numeric(filtered_data_insecurity['Household Insecurity'], errors='coerce')

# Merge the two datasets on State to align counties correctly
merged_food_insecurity_low_access = filtered_data_insecurity.merge(state_avg_low_access, on='State', how='left')

# Create scatter plot
plt.figure(figsize=(8, 6))
sns.scatterplot(
    data=merged_food_insecurity_low_access,
    x='Low Access Population',
    y='Household Insecurity',
    # color='dodgerblue',
    alpha=0.6,  # Reduce opacity for overlap handling
    s=80  # Increase point size for better visibility
)

# Add a trend line
sns.regplot(
    data=merged_food_insecurity_low_access,
    x='Low Access Population',
    y='Household Insecurity',
    scatter=False,  # Hide scatter points for the regression plot
    color='red',
    ci=95,  # Confidence interval (default: 95%)
    line_kws={'linewidth': 2, 'alpha': 0.8}  # Adjust line opacity
)

# Improve labels and title
plt.title("Household Food Insecurity vs. Low Access to Store by State", fontsize=14)
plt.xlabel("Population with Low Access to Store (Raw)", fontsize=12)
plt.ylabel("Household Food Insecurity", fontsize=12)

# Add grid for better readability
plt.grid(True, linestyle="--", alpha=0.5)

plt.show()

In [ ]:
# median household income vs. food insecurity 

# Filter median household income 
filtered_data_income = state_and_county[state_and_county['Variable_Code'] == 'MEDHHINC15'].copy()
filtered_data_income['Median Household Income'] = pd.to_numeric(filtered_data_income['Value'], errors='coerce')

state_avg_median_income = filtered_data_income.groupby('State')['Median Household Income'].median().reset_index()

# Merge the two datasets on State to align counties correctly
merged_median_income_food_insecurity = filtered_data_insecurity.merge(state_avg_median_income, on='State', how='left')

# Set figure size and style
plt.figure(figsize=(10, 6))
sns.set_style("whitegrid")

# Scatter plot
sns.scatterplot(
    data=merged_median_income_food_insecurity,  
    x='Median Household Income',  
    y='Household Insecurity',  
    alpha=0.6,  # Reduce opacity for overlap handling
    s=80  # Increase point size for better visibility
)

# Add regression line
sns.regplot(
    data=merged_median_income_food_insecurity,  
    x='Median Household Income',  
    y='Household Insecurity',  
    scatter=False,  # Hide original points
    line_kws={'color': 'red'}  # Trend line in red
)

# Adjust x-axis labels to thousands (K)
# plt.xticks(ticks=plt.xticks()[0], labels=[f"{int(tick/1000)}K" for tick in plt.xticks()[0]])

# Titles and labels
plt.title("Median Household Income vs. Food Insecurity", fontsize=14)
plt.xlabel("Median Household Income ($)", fontsize=12)
plt.ylabel("Household Food Insecurity (%)", fontsize=12)

# Show plot
plt.show()

In [ ]:
# poverty rate vs. food insecurity 

# Filter median household income 
filtered_data_poverty = state_and_county[state_and_county['Variable_Code'] == 'POVRATE15'].copy()
filtered_data_poverty['Poverty Rate, 2015'] = pd.to_numeric(filtered_data_poverty['Value'], errors='coerce')

state_avg_poverty_rate = filtered_data_poverty.groupby('State')['Poverty Rate, 2015'].mean().reset_index()

# Merge the two datasets on State to align counties correctly
merged_poverty_rate_food_insecurity = filtered_data_insecurity.merge(state_avg_poverty_rate, on='State', how='left')

# Set figure size and style
plt.figure(figsize=(10, 6))
sns.set_style("whitegrid")

# Scatter plot
sns.scatterplot(
    data=merged_poverty_rate_food_insecurity,  
    x='Poverty Rate, 2015',  
    y='Household Insecurity',  
    alpha=0.6,  # Reduce opacity for overlap handling
    s=80  # Increase point size for better visibility
)

# Add regression line
sns.regplot(
    data=merged_poverty_rate_food_insecurity,  
    x='Poverty Rate, 2015',  
    y='Household Insecurity',  
    scatter=False,  # Hide original points
    line_kws={'color': 'red'}  # Trend line in red
)

# Adjust x-axis labels to thousands (K)
# plt.xticks(ticks=plt.xticks()[0], labels=[f"{int(tick/1000)}K" for tick in plt.xticks()[0]])

# Titles and labels
plt.title("Poverty Rate, 2015 vs. Food Insecurity", fontsize=14)
plt.xlabel("Poverty Rate, 2015 (%)", fontsize=12)
plt.ylabel("Household Food Insecurity (%)", fontsize=12)

# Show plot
plt.show()

In [ ]:
# linear regression

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# merge
predictors_df = state_avg_median_income.merge(state_avg_poverty_rate, on='State')
final_df = filtered_data_insecurity.merge(predictors_df, on='State', how='left')

# drop NA 
final_df = final_df.dropna(subset=['Household Insecurity', 'Median Household Income', 'Poverty Rate, 2015'])

# set up 
X = final_df[['Median Household Income', 'Poverty Rate, 2015']]
y = final_df['Household Insecurity']

# fit
model = LinearRegression()
model.fit(X, y)

# predictions
y_pred = model.predict(X)

# eval
mse = mean_squared_error(y, y_pred)
r2 = r2_score(y, y_pred)

# compare
y_baseline = np.full_like(y, np.median(y), dtype=np.float64)
baseline_mse = mean_squared_error(y, y_baseline)
baseline_r2 = r2_score(y, y_baseline)

# results
print("Model MSE:", mse, "R²:", r2)
print("Baseline MSE:", baseline_mse, "R²:", baseline_r2)

In [ ]:
# Function to filter and rename race data
def get_race_data(var_code, race_name):
    df = state_and_county[state_and_county['Variable_Code'] == var_code].copy()
    df[race_name] = pd.to_numeric(df['Value'], errors='coerce')
    return df[['State', race_name]].groupby('State').mean().reset_index()

# Get racial percentages
white_data = get_race_data('PCT_NHWHITE10', 'White')
black_data = get_race_data('PCT_NHBLACK10', 'Black')
hispanic_data = get_race_data('PCT_HISP10', 'Hispanic')
asian_data = get_race_data('PCT_NHASIAN10', 'Asian')
native_american_data = get_race_data('PCT_NHNA10', 'Native American or Alaska Native')
hpi_data = get_race_data('PCT_NHPI10', 'Hawaiian or Pacific Islander')

# Merge all racial data into one DataFrame
state_avg_race = white_data
for df in [black_data, hispanic_data, asian_data, native_american_data, hpi_data]:
    state_avg_race = state_avg_race.merge(df, on='State', how='left')

# Merge with food insecurity data
merged_data_race = filtered_data_insecurity.merge(state_avg_race, on='State', how='left')

# Set figure size and style
fig, axes = plt.subplots(3, 2, figsize=(12, 12))
axes = axes.flatten()
palette = sns.color_palette("husl", 6)

# Standardize Y-axis range
ymin = merged_data_race['Household Insecurity'].min()
ymax = merged_data_race['Household Insecurity'].max()

# Loop through racial groups to create subplots
for i, race in enumerate(['White', 'Black', 'Hispanic', 'Asian', 'Native American or Alaska Native', 'Hawaiian or Pacific Islander']):
    sns.scatterplot(
        data=merged_data_race,
        x=race,
        y='Household Insecurity',
        ax=axes[i],
        color=palette[i],
        alpha=0.6,
        s=80
    )
    
    # Add regression line
    sns.regplot(
        data=merged_data_race,
        x=race,
        y='Household Insecurity',
        scatter=False,
        ax=axes[i],
        line_kws={'color': 'red', 'linewidth': 1.5},
        ci=95
    )

    # Calculate correlation coefficient
    corr, _ = pearsonr(merged_data_race[race].dropna(), merged_data_race['Household Insecurity'].dropna())
    axes[i].text(0.05, 0.9, f'r = {corr:.2f}', transform=axes[i].transAxes, fontsize=10, color='gray')

    # Standardize y-axis range
    axes[i].set_ylim(ymin, ymax)

    # Improve titles and labels
    axes[i].set_title(f'Food Insecurity vs.\n{race} (%)', fontsize=11)
    axes[i].set_xlabel(f'{race} Population (%)')


plt.tight_layout()
plt.show()